In [1]:
import spotipy
import webbrowser
import json
import urllib.request
import geocoder
from nltk.corpus import stopwords
import spotipy.util as util
import random

In [2]:
# copy and paste the url into your browser to see what data you are getting back
request = urllib.request.Request("https://quandyfactory.com/insult/json")
response = urllib.request.urlopen(request)

In [3]:
#checking the type of my response
type(response)

http.client.HTTPResponse

In [4]:
# converting this HTTPresponse into its own dictionary
insults = json.loads(response.read())

In [5]:
insults

{'insult': 'Thou art a Wayward, Dog-hearted Huggermugger.',
 'source': 'The Elizabethan Insult Generator',
 'sourceUrl': 'http://www.museangel.net/insult.html#generator'}

In [6]:
# extracting the keys from the json file that we want to use as our prompt to create spotify playlist
insult = insults['insult']

In [7]:
# I want to go through each word of each title
for word in insult.split(" "):
    print(word)
    # .split(" "): will prevent python from merely printing each character

Thou
art
a
Wayward,
Dog-hearted
Huggermugger.


In [8]:
# creating a variable which contains the list of words from the insult
list_of_words = insults['insult'].split(" ")

In [9]:
print(list_of_words)
# these are what I want to effectively have as each song of my playlist to spell the insult through an acrostic

['Thou', 'art', 'a', 'Wayward,', 'Dog-hearted', 'Huggermugger.']


In [10]:
# open file with keys and set the path to your credentials JSON file
credentials = "spotify_keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [11]:
# read the keys and assign each to a variable
# A redirect URI, or reply URL, is the location where the authorization server sends the user once the app has been 
# successfully authorized and granted an authorization code or access token.
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

In [12]:
# scope is requesting special permissions to make playlist from spotify
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-private user-library-read playlist-read-private app-remote-control'
token = util.prompt_for_user_token(username, scope, client_id=client_id,client_secret=client_secret,redirect_uri=redirectURI)

In [13]:
# create spotify object
# this is how we will interact with our api
sp = spotipy.Spotify(auth=token)

In [18]:
songs = []
#for every word in list of words
for search_term in list_of_words:
    search_results = sp.search(q="track:" + search_term, type="track")
    # if the search returns anything
    if len(search_results['tracks']['items']) > 0:
        first_song = search_results['tracks']['items'][0]
        songs.append(first_song['uri'])

In [21]:
songs

['spotify:track:4w1lzcaoZ1IC2K5TwjalRP',
 'spotify:track:1SvIUtl5QuOTJhKNVikt6m',
 'spotify:track:39JofJHEtg8I4fSyo7Imft',
 'spotify:track:4DMKwE2E2iYDKY01C335Uw',
 'spotify:track:3MSDoAW4TWSJQZNVHKBDZg',
 'spotify:track:0HY8Uzo6BBe5ugKEhT2ezd']

In [27]:
my_playlist = sp.user_playlist_create(user=username, name="insult from an elizabethan", public=False,
                                      description="spotify playlist made with elizabethan insult api")
results = sp.user_playlist_add_tracks(username, my_playlist['id'], songs)
print(results)

HTTP Error for POST to https://api.spotify.com/v1/users/1120584804/playlists with Params: {} returned 401 due to The access token expired


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/users/1120584804/playlists:
 The access token expired, reason: None

In [28]:
webbrowser.open(my_playlist['external_urls']['spotify'])

True